In [ ]:
#!/usr/bin/python
import os
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy
import sys
import configparser
import progressbar
import pandas as pd
from os.path import expanduser

sys.path.append(os.path.realpath('.'))
home = expanduser("~")

#Parse the config parameters
Config=configparser.ConfigParser()
Config.read(home+'/'+'hesp.config')
objname=Config.get('Headers','objname')
objtype=Config.get('Headers','objtype')
reseltype=Config.get('Headers','reseltype')
dateobs=Config.get('Headers','dateobs')
timeobs=Config.get('Headers','timeobs')
exptime=Config.get('Headers','exptime')
reselkeylow=Config.get('Keys','reselkeylow')
reselkeyhigh=Config.get('Keys','reselkeyhigh')
biaskey=Config.get('Keys','biaskey')
flatkey=Config.get('Keys','flatkey')
calibkey=Config.get('Keys','calibkey')
objectkey=Config.get('Keys','objectkey')
fits_index=int(Config.get('Fits','index'))
binsize=float(Config.get('Extraction','binsize'))
binres=float(Config.get('Extraction','binres'))
path=Config.get('Config','path')
directory=Config.get('Headers','directory')

#Open the filelist
f = open(directory+'files.txt', 'w')

#Lists for storing the filetypes
datlist=[]
bl=[]
ol=[]
objl=[]
tharl=[]
coloh=[]
tracel=[]

#Display

#Calculate number of files !Should rewrite with shell code
numfile=0
for fn in os.listdir(directory):
    if os.path.isfile(directory+fn):
        if fn.endswith('fits'):
               numfile=numfile+1

#Start Loop
a=0
print(  "'Analysing Directory...' \n" )
bar = progressbar.ProgressBar(maxval=numfile, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage(),' ', progressbar.ETA()])
bar.start()
for fn in os.listdir(directory):
     if os.path.isfile(directory+fn):
        if fn.endswith('fits'):
           hdulist = fits.open(directory+fn)

           # Read data block
           try :
             scidata=hdulist[1].data
           except:
             scidata=hdulist[0].data
           if len(scidata.shape)==3:
             scidata=scidata[0]

           #Read headers and classify. Read data and compute mean,std etc..
           try:
               val=hdulist[0].header[objtype]
               date=str(hdulist[0].header[dateobs])
               time=str(hdulist[0].header[timeobs])
               dat=hdulist[0].header[dateobs]
               exp=str(hdulist[0].header[exptime])
               a=a+1
               bar.update(a)
               if val==biaskey:
                   fn=fn.ljust(25)
                   mn='{0:.2f}'.format(scidata.mean())
                   rms='{0:.2f}'.format(scidata.std())
                   bl.append(fn.ljust(30)+mn.ljust(12)+rms.ljust(12)+date.ljust(17)+time.ljust(17))
                   coloh.append(1)
               elif val==flatkey:
                   fn=fn.ljust(20)
                   resel=hdulist[0].header[reseltype]
                   if resel==reselkeylow:
                       reselname='Low'
                   else:
                       reselname='High'
                   mn='{0:.2f}'.format(scidata.mean())
                   md='{0:.2f}'.format(numpy.median(scidata))
                   objl.append(hdulist[0].header[objname])
                   tracel.append(fn.ljust(30)+mn.ljust(12)+exp.ljust(12)+reselname.ljust(12)+date.ljust(17)+time.ljust(17))

               elif val==calibkey:
                   fn=fn.ljust(20)
                   objct=hdulist[0].header[objname].replace(" ", "_")
                   resel=hdulist[0].header[reseltype]
                   if resel==reselkeylow:
                       reselname='Low'
                   else:
                       reselname='High'
                   mn='{0:.2f}'.format(scidata.mean())
                   md='{0:.2f}'.format(numpy.median(scidata))
                   objl.append(hdulist[0].header[objname].replace(" ", "_"))
                   tharl.append(fn.ljust(30)+mn.ljust(12)+exp.ljust(12)+reselname.ljust(12)+date.ljust(17)+time.ljust(17))

               elif val==objectkey:
                   fn=fn.ljust(20)
                   objct=hdulist[0].header[objname].replace(" ", "_")
                   resel=hdulist[0].header[reseltype]
                   if resel==reselkeylow:
                       reselname='Low'
                   else:
                       reselname='High'
                   mn='{0:.2f}'.format(scidata.mean())
                   md='{0:.2f}'.format(numpy.median(scidata))
                   objl.append(hdulist[0].header[objname].replace(" ", "_"))
                   ol.append(fn.ljust(30)+objct.ljust(15)+mn.ljust(12)+exp.ljust(12)+reselname.ljust(12)+date.ljust(17)+time.ljust(17))
                   coloh.append(5)
           except KeyError:
                pass
bar.finish()
f.write('BIAS\n')
f.write('File      '.ljust(30)+'Mean'.ljust(12)+'RMS'.ljust(12)+'Date'.ljust(17)+'Time'.ljust(17))
f.write('\n')
for l in bl:
        f.write(l)
        f.write('\n')
f.write('\n')
f.write('OBJECT\n')
f.write('File'.ljust(30)+'Object'.ljust(15)+'Mean'.ljust(12)+'Exposure'.ljust(12)+'Resolution'.ljust(12)+'Date'.ljust(17)+'Time'.ljust(17))
f.write('\n')

for l in ol:
        f.write(l)
        f.write('\n')
f.write('\n')
f.write('FLAT\n')
f.write('File'.ljust(30)+'Mean'.ljust(12)+'Exposure'.ljust(12)+'Resolution'.ljust(12)+'Date'.ljust(17)+'Time'.ljust(17))
f.write('\n')

for l in tracel:
        f.write(l)
        f.write('\n')
f.write('\n')
f.write('CALIB\n')
f.write('File'.ljust(30)+'Mean'.ljust(12)+'Exposure'.ljust(12)+'Resolution'.ljust(12)+'Date'.ljust(17)+'Time'.ljust(17))
f.write('\n')

for l in tharl:
        f.write(l)
        f.write('\n')

f.close()






: 

### `hesp_preproc` : Preprocessing of the files including, Bias Subtraction, Overscan Correction, Trimming and Cosmic Ray Correction

In [2]:
#!/usr/bin/python
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
import numpy, time
import numpy as np
import peakutils
from astropy.io import fits
from scipy.ndimage import morphology as morph
from scipy.ndimage import filters
import os
import progressbar
from time import sleep 
def biascombine(files, output='BIAS.fit', trim=False, silent=False):
    # assume biaslist is a simple text file with image names
    # e.g. ls flat.00*b.fits > bflat.lis    
    if silent is False:
        print('biascombine: combining ' + str(len(files)) + ' files')

    for i in range(0,len(files)):
        hdu_i = fits.open(files[i][:-5]+'_pp.fit',do_not_scale_image_data=True)

        if trim is False:
            im_i = hdu_i[0].data
        if trim is True:
            datasec = hdu_i[0].header['DATASEC'][1:-1].replace(':',',').split(',')
            d = map(float, datasec)
            im_i = hdu_i[0].data[d[2]-1:d[3],d[0]-1:d[1]]

        # create image stack
        if (i==0):
            all_data = np.array(im_i)
        elif (i>0):
            all_data = np.dstack( (all_data, im_i) )
        hdu_i.close(closed=True)

    # do median across whole stack
    #print all_data
    try:
        bias = np.median(all_data, axis=2)
    except:
        bias= np.array(all_data)        
    #print bias.shape
    # write output to disk for later use
    hduOut = fits.PrimaryHDU(bias)
    hduOut.writeto(output, overwrite=True)
     
def overscanbias(img, cols1=(1,), cols2=(1,)):
    '''
    Generate a bias frame based on overscan region.
    Can work with rows or columns, pass either kwarg the limits:
    >>> bias = overscanbias(imagedata, cols=(1024,1050))
    '''
    bias = np.zeros_like(img)
    if len(cols1) > 1:
        bcol = np.mean(img[:,cols1[0]:cols1[1]], axis=1)
        for j in range(img.shape[1]):
            img[:,j] = bcol

    if len(cols2) > 1:
        brow = np.mean(img[:,cols2[0]:cols2[1]], axis=1)
        for j in range(img.shape[1]):
              bias[:,j] = brow
              img[:,j]=(img[:,j]+bias[:,j])/2
          
    if len(cols1)==1:
        print('OVERSCANBIAS ERROR: need to pass either cols=(a,b) or rows=(a,b),')
        print('setting bias = zero as result!')

    return img
        



### `hesp_extract`: Extract the orders from the processed files. This command will extract all the files listed in files.txt

In [3]:
#!/usr/bin/python
import configparser
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
import numpy, time
import numpy as np
from astropy.io import fits
import os
import progressbar
from hesp_lib import *
from ccdproc import  *
import pandas as pd
import copy
import progressbar
from scipy import interpolate
from os.path import expanduser
import sys
import configparser
home = expanduser("~")

#Parse the config parameters
Config=configparser.ConfigParser()
Config.read(home+'/'+'hesp.config')
objname=Config.get('Headers','objname')
objtype=Config.get('Headers','objtype')
reseltype=Config.get('Headers','reseltype')
dateobs=Config.get('Headers','dateobs')
exptime=Config.get('Headers','exptime')
reselkeylow=Config.get('Keys','reselkeylow')
reselkeyhigh=Config.get('Keys','reselkeyhigh')
biaskey=Config.get('Keys','biaskey')
flatkey=Config.get('Keys','flatkey')
calibkey=Config.get('Keys','calibkey')
objectkey=Config.get('Keys','objectkey')
fits_index=int(Config.get('Fits','index'))
binsize=float(Config.get('Extraction','binsize'))
binres=float(Config.get('Extraction','binres'))
path=Config.get('Config','path')
biasflg=Config.get('Process','bias')
ovrscnflg=Config.get('Process','overscan')
csmcflg=Config.get('Process','cosmic')
csmc_clip=Config.get('Process','csmc_clip')
directory=Config.get('Headers','directory')
#Read the file list
fl = open(directory+'files.txt', 'r')

exfl=sys.argv[1]
files=[exfl]

for i in files:
 print 'Extracting File '+ i   
 hdu= fits.open(i)
 scidata=hdu[0].data	 
 xaxis=numpy.linspace(0,4112,4112)
 yaxis=numpy.linspace(0,4096,4096)
 xx,yy=numpy.meshgrid(xaxis,yaxis)
 f=interpolate.RectBivariateSpline(xaxis, yaxis, scidata)
 k=numpy.array([])
 data=numpy.array([])
 data2=numpy.array([])
 scattery=numpy.array([])
 
 try :
	 resel=hdu[0].header[reseltype]
 except:
	 print 'Failed extracting Resolution from header..'
	 print 'Assuming Low Resolution'
	 resel='0'

 if resel==reselkeylow:
     fl = path+'lowresfiber1.trace'
     content=np.loadtxt(fl)
     a=0
     bar = progressbar.ProgressBar(maxval=len(content), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage(),' ', progressbar.ETA()])
     print  "Extracting Orders from fiber 1... \n"
     bar.start()
     for l in range(content.shape[0]):
	 
      bar.update(a)     
      cp=content[l]
      p=np.poly1d(cp)
   
      
      datadummy=numpy.array([])
      for x0 in range(0,4096,1):
       scatbiny=[]
       scatbinx=[] 
       y0=p(x0)
       sum=0
       for pix in numpy.arange(y0-(binsize/2),y0+(binsize/2),binres):
	scatbiny=numpy.append(scatbiny,pix)
	y=pix
	#dydx=2*p[0]*x0+p[1]
	#x=(-dydx)*(y-y0)+x0 In case of normal binning
	x=x0
	scatbinx=numpy.append(scatbinx,x)
	sum=sum+f(y,x)
       datadummy=numpy.append(datadummy,sum) 
      if a==0:
	data= numpy.hstack((data,datadummy))
      else: 
	data= numpy.vstack((data,datadummy))
      a=a+1  
     bar.finish()	 
     print  "Extraction Complete. \n"   
     hduOut =fits.PrimaryHDU(data)
     hduOut.header=hdu[0].header
     hduOut.writeto(i[:-7]+'_fiber1_ec.fit',overwrite=True)


     data=numpy.array([])
     fl = path+'lowresfiber2.trace'
     content=np.loadtxt(fl)

     a=0
     bar = progressbar.ProgressBar(maxval=len(content), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage(),' ', progressbar.ETA()])
     print  "Extracting Orders from fiber 2... \n"
     bar.start()
     for l in range(content.shape[0]):

      bar.update(a)     
      cp=content[l]
      p=np.poly1d(cp)
      
      datadummy=numpy.array([])
      for x0 in range(0,4096,1):
       scatbiny=[]
       scatbinx=[] 
       y0=p(x0)
       sum=0
       for pix in numpy.arange(y0-(binsize/2),y0+(binsize/2),binres):
	scatbiny=numpy.append(scatbiny,pix)
	y=pix
	#dydx=2*p[0]*x0+p[1]
	#x=(-dydx)*(y-y0)+x0
	x=x0
	scatbinx=numpy.append(scatbinx,x)
	sum=sum+f(y,x)
       datadummy=numpy.append(datadummy,sum) 

      if a==0:
	data= numpy.hstack((data,datadummy))
      else: 
	data= numpy.vstack((data,datadummy))
      a=a+1  
     bar.finish()	 
     print  "Extraction Complete. \n"   
     plt.show()  
     hduOut =fits.PrimaryHDU(data)
     hduOut.header=hdu[0].header
     hduOut.writeto(i[:-7]+'_fiber2_ec.fit',overwrite=True)
 if resel==reselkeyhigh:
     
     fl = path+'highresfiber1.trace'
     content=np.loadtxt(fl)
     
     
     a=0
     bar = progressbar.ProgressBar(maxval=len(content), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage(),' ', progressbar.ETA()])
     print  "Extracting Orders from fiber 1... \n"
     bar.start()
     for l in range(content.shape[0]):
	 
      bar.update(a)     
      cp=content[l]
      p=np.poly1d(cp)
      
      
      datadummy=numpy.array([])
      for x0 in range(0,4096,1):
       scatbiny=[]
       scatbinx=[] 
       y0=p(x0)
       sum=0
       for pix in numpy.arange(y0-(binsize/2),y0+(binsize/2),binres):
	scatbiny=numpy.append(scatbiny,pix)
	y=pix
	#dydx=2*p[0]*x0+p[1]
	#x=(-dydx)*(y-y0)+x0 In case of normal binning
	x=x0
	scatbinx=numpy.append(scatbinx,x)
	sum=sum+f(y,x)
       datadummy=numpy.append(datadummy,sum) 
      if a==0:
	data= numpy.hstack((data,datadummy))
      else: 
	data= numpy.vstack((data,datadummy))
      a=a+1  
     bar.finish()	 
     print  "Extraction Complete. \n"   
     hduOut =fits.PrimaryHDU(data)
     hduOut.header=hdu[0].header
     hduOut.writeto(i[:-7]+'_fiber1_ec.fit',overwrite=True)


     data=numpy.array([])
     fl = path+'highresfiber2.trace'
     content=np.loadtxt(fl)


     a=0
     bar = progressbar.ProgressBar(maxval=len(content), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage(),' ', progressbar.ETA()])
     print  "Extracting Orders from fiber 2... \n"
     bar.start()
     for l in range(content.shape[0]):
      bar.update(a)
      sleep(0.1)
      cp=content[l]
      p=np.poly1d(cp)


      datadummy=numpy.array([])
      for x0 in range(0,4096,1):
       scatbiny=[]
       scatbinx=[] 
       y0=p(x0)
       sum=0
       for pix in numpy.arange(y0-(binsize/2),y0+(binsize/2),binres):
	scatbiny=numpy.append(scatbiny,pix)
	y=pix
	#dydx=2*p[0]*x0+p[1]
	#x=(-dydx)*(y-y0)+x0
	x=x0
	scatbinx=numpy.append(scatbinx,x)
	sum=sum+f(y,x)
       datadummy=numpy.append(datadummy,sum) 

      if a==0:
	data= numpy.hstack((data,datadummy))
      else: 
	data= numpy.vstack((data,datadummy))
      a=a+1  
     bar.finish()	 
     print  "Extraction Complete. \n"   
     plt.show()  
     hduOut =fits.PrimaryHDU(data)
     hduOut.header=hdu[0].header
     hduOut.writeto(i[:-7]+'_fiber2_ec.fit',overwrite=True)   
 
 


SyntaxError: Missing parentheses in call to 'print'. Did you mean print('Extracting File '+ i)? (3523679093.py, line 52)